# Basic Optimization
In this notebook I'll optimize a trough model to fit the data.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import mars_troughs as mt
print("Creating a trough object.")
test_acc_params = [1e-8, 5e-9]
acc_model_number = 1
test_lag_params = [1, 1e-5]
lag_model_number = 1
errorbar = 100.
tr = mt.Trough(test_acc_params, test_lag_params,
                acc_model_number, lag_model_number,
                errorbar)

Creating a trough object.
